In [1]:
include("../src/TensorDecomposition.jl")
using LinearAlgebra, Combinatorics, LinearSolve

We demonstrate that this algorithm works for larger $d$ even, such as $d=6$.  We set $n=5, r=50$, and draw $r$ generic points in $\mathbb{R}^{6}$, constructing the corresponding tensor $\phi\in S^6\mathbb{R}^{6}$.  We remark that because the algorithm is given numerically the problem may be ill-conditioned; robustness of the linear extension procedure merits further consideration.

In [2]:
n = 5
r = binomial(5+2, 2)+29

D, Drev = TensorDecomposition.makeDicts(n, 6);
basis_inds = collect(1:r)
basis, basisD = TensorDecomposition.basisFn(basis_inds, Drev);

vars = TensorDecomposition.varTups(basis, n, 6)
eqs1, eqs2 = TensorDecomposition.linEqTups(basisD, n, 6);

r

50

In [12]:
Z = randn(n+1, r)
T = TensorDecomposition.rankedTensor(ones(r), Z, 6; type=eltype(Z));

Tcat = TensorDecomposition.catMat(T, 3)
H0 = Tcat[basis_inds, basis_inds]

Z_ = Z ./ permutedims(Z[1, :]);
permutedims(Z_)

50×6 Matrix{Float64}:
 1.0    0.46318     0.591717   -0.447893     0.539957   -0.294486
 1.0    0.0589931   0.608538    1.193       -0.0546005   1.55941
 1.0   -5.82562     8.80492    11.4615      19.5018     -7.30224
 1.0   -1.66452    11.9812      6.77257     -4.76822     6.18836
 1.0    0.927926   -0.808093   -0.220273     0.729999   -0.113598
 1.0   -0.29308     0.929967    1.78933      0.810826   -0.0295807
 1.0   -1.45077     1.8904     -0.0290537   -0.379407    1.10767
 1.0   -0.266586    0.563335   -0.138551    -0.0976242   0.19679
 1.0    2.79142     2.73129    -0.798674    -0.901155    0.502134
 1.0  -15.6483     -3.90698   -12.7808      -3.29807    14.4643
 ⋮                                                       ⋮
 1.0    1.00211    -0.425453    0.130478    -0.0850527   0.449713
 1.0   -2.5132     -0.886726    1.58008      1.52786     0.380648
 1.0    0.790474    1.61291    -1.23877      0.933341    0.907007
 1.0    0.769544    0.904588   -2.734        4.50751     3.5501
 1.

In [13]:
A, b = TensorDecomposition.linearSystem(T, H0, basis_inds, basisD, D, vars, eqs1, eqs2; type=eltype(T));

A_ = Matrix(copy(A))
foreach(TensorDecomposition.normalize!, eachcol(A_));
foreach(TensorDecomposition.normalize!, eachrow(A_));
svdvals(A_)

291-element Vector{Float64}:
 3.1543039295740205
 3.0071672368781304
 2.946748253866847
 2.843086642258858
 2.7276457361014947
 2.621879152811944
 2.60857364203648
 2.575763295764647
 2.517972199380121
 2.4846598616436975
 ⋮
 0.002393343659184399
 0.0020540129598886963
 0.0017474498818730623
 0.0008425306910932216
 0.0007227521749637421
 0.0005797299662867797
 0.0003517792780250445
 0.0002498032758918743
 0.00013085662744829088

In [14]:
size(A)

(341, 291)

The linear system matrix $\mathbf{A}$ is $341\times 291$, and we can determine that it is full column rank.

In [15]:
prob = LinearProblem(A, b)
sol = solve(prob)
solDict = Dict([v => s for (v, s) in zip(vars, sol.u)]);
Ms = TensorDecomposition.multMatrices(T, basis, solDict, D, H0)
lhat, Zhat = TensorDecomposition.obtainDecomp(T, Ms);

In [16]:
maximum(abs.(T-TensorDecomposition.rankedTensor(lhat, Zhat, 6, type=eltype(Zhat))))/maximum(abs.(T))

2.2525866946947747e-9

Now we let $\mathbf{z}_{50}=\mathbf{z}_1+\mathbf{z}_2$, and leave $\mathbf{z}_3, \dots, \mathbf{z}_{r-1}$ generic; we form the corresponding tensor $\phi$.

In [17]:
Z[:, r] = Z[:, 1] + Z[:, 2]
T = TensorDecomposition.rankedTensor(ones(r), Z, 6; type=eltype(Z));

Tcat = TensorDecomposition.catMat(T, 3)
H0 = Tcat[basis_inds, basis_inds]

A, b = TensorDecomposition.linearSystem(T, H0, basis_inds, basisD, D, vars, eqs1, []; type=eltype(T));

A_ = Matrix(copy(A))
foreach(TensorDecomposition.normalize!, eachcol(A_));
foreach(TensorDecomposition.normalize!, eachrow(A_));
svdvals(A_)

291-element Vector{Float64}:
 3.1866180607286667
 3.0327799541369638
 2.707034428378973
 2.6413707656429986
 2.5240295838346984
 2.50062456980039
 2.444476027674599
 2.3988007424748394
 2.3741922671801046
 2.2902084474882445
 ⋮
 0.0005199631223893145
 0.00036257248850748937
 0.00031901954000738606
 0.0001688729468158448
 0.00012718504864881936
 6.204119256542351e-5
 4.053485026056819e-5
 1.9642957867055648e-5
 8.100225372534683e-6

The matrix $\mathbf{A}$ is still full column rank; in contrast to $d=4$, order-$6$ tensors with a decomposition with three colinear points are identifiable.

In [18]:
prob = LinearProblem(A, b)
sol = solve(prob)
solDict = Dict([v => s for (v, s) in zip(vars, sol.u)]);
Ms = TensorDecomposition.multMatrices(T, basis, solDict, D, H0)
lhat, Zhat = TensorDecomposition.obtainDecomp(T, Ms);

In [19]:
maximum(abs.(T-TensorDecomposition.rankedTensor(lhat, Zhat, 6, type=eltype(Zhat))))/maximum(abs.(T))

1.602835575937099e-9

Now we consider the tensor $\phi=x_0^3x_1x_2x_3+x_0x_1^3x_2x_3+x_0x_1x_2^3x_3+x_0x_1x_2x_3^3$.

In [25]:
n = 3

inds = [
    [1, 1, 1, 2, 3, 4],
    [1, 2, 2, 2, 3, 4],
    [1, 2, 3, 3, 3, 4],
    [1, 2, 3, 4, 4, 4]
]
T = zeros(4, 4, 4, 4, 4, 4)
for ind in inds
    for perm in Combinatorics.permutations(ind)
        T[perm...] = 1
    end
end

D, Drev = TensorDecomposition.makeDicts(n, 6);

In [26]:
for k=0:6
    rk = rank(TensorDecomposition.catMat(T, k))
    println("$k: $rk")
end

0: 1
1: 4
2: 7
3: 8
4: 7
5: 4
6: 1


As we see, the ranks of the catalecticants are not maximal at each $k$.  In particular, the ranks of the second/fifth catalecticants are expected to be $\min\{r, \binom{n+2}{n}\}$, where $r$ is the rank of the tensor.  We find a set $B$ algorithmically.

In [27]:
basis_inds = TensorDecomposition.findB(T);
basis, basisD = TensorDecomposition.basisFn(basis_inds, Drev);

In [28]:
basis

8-element Vector{Vector{Int64}}:
 [0, 0, 0]
 [1, 0, 0]
 [0, 1, 0]
 [0, 0, 1]
 [1, 1, 0]
 [1, 0, 1]
 [0, 1, 1]
 [1, 1, 1]

In [29]:
vars = TensorDecomposition.varTups(basis, n, 6)
eqs1, eqs2 = TensorDecomposition.linEqTups(basisD, n, 6);

In [ ]:
Tcat = TensorDecomposition.catMat(T, 3)
H0 = Tcat[basis_inds, basis_inds]

A, b = TensorDecomposition.linearSystem(T, H0, basis_inds, basisD, D, vars, eqs1, eqs2; type=eltype(T));

svdvals(Matrix(copy(A)))

3-element Vector{Float64}:
 1.4142135623730951
 1.4142135623730951
 1.414213562373095

In [31]:
size(A)

(9, 3)

There are more linearly independent linear equations than moment variables.

In [32]:
prob = LinearProblem(A, b)
sol = solve(prob)
solDict = Dict([v => s for (v, s) in zip(vars, sol.u)]);
Ms = TensorDecomposition.multMatrices(T, basis, solDict, D, H0)
lhat, Zhat = TensorDecomposition.obtainDecomp(T, Ms);

In [33]:
permutedims(Zhat)

8×4 Matrix{Float64}:
 1.0   1.0   1.0   1.0
 1.0  -1.0   1.0   1.0
 1.0   1.0  -1.0   1.0
 1.0  -1.0  -1.0   1.0
 1.0   1.0   1.0  -1.0
 1.0  -1.0   1.0  -1.0
 1.0   1.0  -1.0  -1.0
 1.0  -1.0  -1.0  -1.0

In [34]:
maximum(abs.(T-TensorDecomposition.rankedTensor(lhat, Zhat, 6, type=eltype(Zhat))))/maximum(abs.(T))

9.020562075079397e-15

This example is chosen to be a specific degree-$6$ extension of the monomial $x_0x_1x_2x_3$.  The monomial does not have a unique decomposition, but the choice we have made in the extension results in a unique rank-$8$ decomposition -- indeed, it is the "canonical" choice, where $\mathbf{z}_{i, j}$ range over the second roots of unity $\pm 1$. 